In [0]:
fp = r'/FileStore/tables/us_500.csv'
sampleEmployee = spark.read.csv(fp, header=True)

In [0]:
display(sampleEmployee.take(5))

first_name,last_name,company_name,address,city,county,state,zip,phone1,phone2,email,web
James,Butt,"Benton, John B Jr",6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com,http://www.bentonjohnbjr.com
Josephine,Darakjy,"Chanay, Jeffrey A Esq",4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org,http://www.chanayjeffreyaesq.com
Art,Venere,"Chemel, James L Cpa",8 W Cerritos Ave #54,Bridgeport,Gloucester,NJ,08014,856-636-8749,856-264-4130,art@venere.org,http://www.chemeljameslcpa.com
Lenna,Paprocki,Feltz Printing Service,639 Main St,Anchorage,Anchorage,AK,99501,907-385-4412,907-921-2010,lpaprocki@hotmail.com,http://www.feltzprintingservice.com
Donette,Foller,Printing Dimensions,34 Center St,Hamilton,Butler,OH,45011,513-570-1893,513-549-4561,donette.foller@cox.net,http://www.printingdimensions.com


In [0]:
from pyspark.sql import functions as F
#Incr. the data by 100X
employeeDF = sampleEmployee.withColumn("city", F.explode(F.array_repeat("city",100)))

In [0]:
vaccine_perday = 100
CityEmployeeDensity = employeeDF.groupBy('city').count().withColumnRenamed("count","pdensity").orderBy('pdensity', ascending=False)
VaccinationDrivePlan = employeeDF.join(CityEmployeeDensity, employeeDF.city == CityEmployeeDensity.city, "inner" )
vaccineSchedule = CityEmployeeDensity.withColumn("total_days", F.ceil(F.col("pdensity")/vaccine_perday)).withColumnRenamed("pdensity", "day_0")
display(vaccineSchedule.take(5))

city,day_0,total_days
New York,1400,14
Philadelphia,800,8
Chicago,700,7
Miami,600,6
San Francisco,500,5


In [0]:
## generate Schedule for vaccination
d = vaccineSchedule.take(1)[0].total_days
for i in range(1,d+1):
  vaccineSchedule = vaccineSchedule.withColumn(f"day_{i}", F.greatest(F.lit(0), (F.col(f"day_{i-1}")-vaccine_perday)))

In [0]:
vaccineSchedule.show()

+-------------+-----+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+
|         city|day_0|total_days|day_1|day_2|day_3|day_4|day_5|day_6|day_7|day_8|day_9|day_10|day_11|day_12|day_13|day_14|
+-------------+-----+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+
|     New York| 1400|        14| 1300| 1200| 1100| 1000|  900|  800|  700|  600|  500|   400|   300|   200|   100|     0|
| Philadelphia|  800|         8|  700|  600|  500|  400|  300|  200|  100|    0|    0|     0|     0|     0|     0|     0|
|      Chicago|  700|         7|  600|  500|  400|  300|  200|  100|    0|    0|    0|     0|     0|     0|     0|     0|
|        Miami|  600|         6|  500|  400|  300|  200|  100|    0|    0|    0|    0|     0|     0|     0|     0|     0|
|      Phoenix|  500|         5|  400|  300|  200|  100|    0|    0|    0|    0|    0|     0|     0|     0|     0|     0|
|San Francisco|  500|   